In [8]:
import numpy as np


In [19]:
annotation_path = "/home/ashish/Documents/interviews/applix/results.npy"
ann = np.load(annotation_path)

In [23]:
print(ann.shape)
print(ann[0])

ann_dict = {}
for i in ann:
    x,y,w,h = [int(i[k]) for k in range(2,6)]
    
    x2,y2 = x + w, y+ h
    if i[0] in ann_dict.keys():
        ann_dict[i[0]].append([x,y,x2,y2])
    else:
        ann_dict[i[0]] = [[x,y,x2,y2]]

print(ann_dict)

(136, 8)
['vid_629_288.jpg' 'frame_78_crop_0.png' '1080' '420' '227' '263'
 '0.7222222222222222' '0.9125988483428955']
{'vid_629_288.jpg': [[1080, 420, 1307, 683]], 'vid_630_54.jpg': [[312, 12, 387, 105], [312, 18, 409, 119]], 'vid_632_504.jpg': [[848, 27, 916, 88], [842, 38, 896, 95]], 'vid_630_612.jpg': [[295, 208, 388, 301], [554, 889, 644, 990], [308, 214, 391, 307], [347, 401, 430, 484]], 'vid_631_42.jpg': [[264, 360, 311, 415], [232, 488, 287, 555]], 'vid_631_18.jpg': [[240, 326, 287, 381]], 'vid_632_42.jpg': [[345, 380, 403, 447], [280, 536, 347, 618]], 'vid_630_72.jpg': [[246, 58, 343, 159]], 'vid_629_84.jpg': [[873, 453, 1135, 757]], 'vid_630_588.jpg': [[607, 880, 740, 992]], 'vid_629_258.jpg': [[649, 305, 932, 571]], 'vid_630_660.jpg': [[551, 140, 646, 219], [538, 146, 625, 233], [534, 29, 613, 99], [527, 41, 589, 107]], 'vid_629_24.jpg': [[859, 426, 1142, 726]], 'vid_629_294.jpg': [[1089, 552, 1296, 772]], 'vid_629_276.jpg': [[868, 468, 1130, 772]], 'vid_633_0.jpg': [[61, 23

In [27]:
def calculate_iou(boxA, boxB):
    # Determine the coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # Compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)

    # Compute the area of both the prediction and ground-truth rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # Compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the intersection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou

def average_boxes(boxA, boxB):
    return [
        int((boxA[0] + boxB[0]) / 2),
        int((boxA[1] + boxB[1]) / 2),
        int((boxA[2] + boxB[2]) / 2),
        int((boxA[3] + boxB[3]) / 2)
    ]

for key, boxes in ann_dict.items():
    i = 0
    while i < len(boxes):
        j = i + 1
        while j < len(boxes):
            if calculate_iou(boxes[i], boxes[j]) > 0.30:
                # Replace with average bounding box
                boxes[i] = average_boxes(boxes[i], boxes[j])
                del boxes[j]
            else:
                j += 1
        i += 1



In [28]:
ann_dict

{'vid_629_288.jpg': [[1080, 420, 1307, 683]],
 'vid_630_54.jpg': [[312, 15, 398, 112]],
 'vid_632_504.jpg': [[845, 32, 906, 91]],
 'vid_630_612.jpg': [[301, 211, 389, 304],
  [554, 889, 644, 990],
  [347, 401, 430, 484]],
 'vid_631_42.jpg': [[264, 360, 311, 415], [232, 488, 287, 555]],
 'vid_631_18.jpg': [[240, 326, 287, 381]],
 'vid_632_42.jpg': [[345, 380, 403, 447], [280, 536, 347, 618]],
 'vid_630_72.jpg': [[246, 58, 343, 159]],
 'vid_629_84.jpg': [[873, 453, 1135, 757]],
 'vid_630_588.jpg': [[607, 880, 740, 992]],
 'vid_629_258.jpg': [[649, 305, 932, 571]],
 'vid_630_660.jpg': [[544, 143, 635, 226], [530, 35, 601, 103]],
 'vid_629_24.jpg': [[859, 426, 1142, 726]],
 'vid_629_294.jpg': [[1089, 552, 1296, 772]],
 'vid_629_276.jpg': [[868, 468, 1130, 772]],
 'vid_633_0.jpg': [[61, 230, 125, 309]],
 'vid_629_162.jpg': [[836, 468, 1056, 684]],
 'vid_629_66.jpg': [[822, 374, 1106, 672]],
 'vid_631_6.jpg': [[242, 319, 289, 374]],
 'vid_631_0.jpg': [[252, 319, 299, 374]],
 'vid_630_642.jpg

In [29]:
template_images = "/home/ashish/Documents/interviews/applix/data/applix_data/training_images"
overlay_path = "/home/ashish/Documents/interviews/applix/data/applix_data/dump/train_data_overlayed"

import cv2
import os

for image_name, boxes in ann_dict.items():
    image_path = os.path.join(template_images, image_name)
    
    if os.path.exists(image_path):
        image = cv2.imread(image_path)
        
        for box in boxes:
            x1, y1, x2, y2 = box
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # cv2.imshow("Image with Bounding Boxes", image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        output_image_path = os.path.join(overlay_path, image_name)
        cv2.imwrite(output_image_path, image)



In [ ]:
/*************  ✨ Codeium Command ⭐  *************/
with open(os.path.join(overlay_path, "annotations.txt"), "w") as f:
    for image_name, boxes in ann_dict.items():
        for box in boxes:
            x1, y1, x2, y2 = box
            f.write("{} {} {} {} {}\n".format(
                image_name, 0, (x1 + x2) / 2, (y1 + y2) / 2, x2 - x1, y2 - y1))
/******  b308a1b1-69c1-44db-93d0-5abd5844fa54  *******/